In [14]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [15]:
# Set a random seed for reproducibility
np.random.seed(93)

In [16]:
# Load the CSV file
csv_path = '/Users/elizabethnemeti/Desktop/masks-dataset/train.csv'
df = pd.read_csv(csv_path)

# Base directories for images and masks
images_base_dir = '/Users/elizabethnemeti/Desktop/uw-madison-gi-tract-image-segmentation/train'
masks_base_dir = '/Users/elizabethnemeti/Desktop/masks-dataset/png/uw-madison-gi-tract-image-segmentation/train'

In [17]:
# Function to check if both image and mask are valid
def is_valid_image_mask_pair(image_path, mask_path):
    if not os.path.exists(image_path) or not os.path.exists(mask_path):
        return False
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path)
    
    if image is None or mask is None:
        return False

    combined_mask = np.zeros(mask.shape[:2], dtype=np.uint8)
    combined_mask[mask[:, :, 0] > 0] = 1  # Large bowel
    combined_mask[mask[:, :, 1] > 0] = 2  # Small bowel
    combined_mask[mask[:, :, 2] > 0] = 3  # Stomach

    # Check if all masks have at least one positive pixel
    if not (np.any(combined_mask == 1) and np.any(combined_mask == 2) and np.any(combined_mask == 3)):
        return False

    return True

# Collect images and masks with a limit
print("Collecting valid image-mask pairs...")
max_images = 2000
valid_count = 0
images = []
masks = []

for index, row in df.iterrows():
    if valid_count >= max_images:
        break
    
    image_relative_path = os.path.relpath(row['image_path'], '/kaggle/input/uw-madison-gi-tract-image-segmentation/train')
    image_path = os.path.join(images_base_dir, image_relative_path)
    mask_relative_path = os.path.relpath(row['mask_path'], '/kaggle/input/uwmgi-mask-dataset/png/uw-madison-gi-tract-image-segmentation/train')
    mask_path = os.path.join(masks_base_dir, mask_relative_path)
    
    if is_valid_image_mask_pair(image_path, mask_path):
        valid_count += 1
        images.append(image_path)
        masks.append(mask_path)
        print(f"Processed {valid_count} valid pairs so far...")

print(f"Total valid image-mask pairs collected: {valid_count}")

Processed 1 valid pairs so far...
Processed 2 valid pairs so far...
Processed 3 valid pairs so far...
Processed 4 valid pairs so far...
Processed 5 valid pairs so far...
Processed 6 valid pairs so far...
Processed 7 valid pairs so far...
Processed 8 valid pairs so far...
Processed 9 valid pairs so far...
Processed 10 valid pairs so far...
Processed 11 valid pairs so far...
Processed 12 valid pairs so far...
Processed 13 valid pairs so far...
Processed 14 valid pairs so far...
Processed 15 valid pairs so far...
Processed 16 valid pairs so far...
Processed 17 valid pairs so far...
Processed 18 valid pairs so far...
Processed 19 valid pairs so far...
Processed 20 valid pairs so far...
Processed 21 valid pairs so far...
Processed 22 valid pairs so far...
Processed 23 valid pairs so far...
Processed 24 valid pairs so far...
Processed 25 valid pairs so far...
Processed 26 valid pairs so far...
Processed 27 valid pairs so far...
Processed 28 valid pairs so far...
Processed 29 valid pairs so f

### PRINTS ALL IMAGES ###

# Function to overlay masks on the image and display the result
def display_image_with_mask(image_path, mask_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Normalize the image to improve visibility
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)

    # Load the mask image
    mask = cv2.imread(mask_path)

    # Check if mask is loaded correctly
    if mask is None:
        return

    # Split the mask into individual channels
    large_bowel_mask = mask[:,:,0]
    small_bowel_mask = mask[:,:,1]
    stomach_mask = mask[:,:,2]

    # Check if all three masks contain non-zero values
    if not (np.any(large_bowel_mask) and np.any(small_bowel_mask) and np.any(stomach_mask)):
        return

    # Create color masks
    large_bowel_color = np.zeros_like(mask)
    small_bowel_color = np.zeros_like(mask)
    stomach_color = np.zeros_like(mask)

    large_bowel_color[:,:,1] = large_bowel_mask  # Green channel for large bowel
    small_bowel_color[:,:,2] = small_bowel_mask  # Blue channel for small bowel
    stomach_color[:,:,0] = stomach_mask  # Red channel for stomach

    # Overlay masks on the original image
    overlay_large_bowel = cv2.addWeighted(cv2.merge([image, image, image]), 0.7, large_bowel_color, 0.3, 0)
    overlay_small_bowel = cv2.addWeighted(cv2.merge([image, image, image]), 0.7, small_bowel_color, 0.3, 0)
    overlay_stomach = cv2.addWeighted(cv2.merge([image, image, image]), 0.7, stomach_color, 0.3, 0)

    # Combine all masks into one overlay
    combined_masks = cv2.addWeighted(large_bowel_color, 0.3, small_bowel_color, 0.3, 0)
    combined_masks = cv2.addWeighted(combined_masks, 1, stomach_color, 0.3, 0)
    overlay_all = cv2.addWeighted(cv2.merge([image, image, image]), 0.7, combined_masks, 0.3, 0)

    # Display the image and masks
    #plt.figure(figsize=(20, 10))

    #plt.subplot(1, 5, 1)
    #plt.title('Image')
    #plt.imshow(image, cmap='gray')
    #plt.axis('off')

    #plt.subplot(1, 5, 2)
    #plt.title('Large Bowel Mask Overlay')
    #plt.imshow(overlay_large_bowel)
    #plt.axis('off')

    #plt.subplot(1, 5, 3)
    #plt.title('Small Bowel Mask Overlay')
    #plt.imshow(overlay_small_bowel)
    #plt.axis('off')

    #plt.subplot(1, 5, 4)
    #plt.title('Stomach Mask Overlay')
    #plt.imshow(overlay_stomach)
    #plt.axis('off')

    #plt.subplot(1, 5, 5)
    #plt.title('All Masks Overlay')
    #plt.imshow(overlay_all)
    #plt.axis('off')

    #plt.show()

# Iterate over the entries in the dataframe
for index, row in df.iterrows():
    # Construct full image path
    image_relative_path = os.path.relpath(row['image_path'], '/kaggle/input/uw-madison-gi-tract-image-segmentation/train')
    image_path = os.path.join(images_base_dir, image_relative_path)

    # Construct full mask path
    mask_relative_path = os.path.relpath(row['mask_path'], '/kaggle/input/uwmgi-mask-dataset/png/uw-madison-gi-tract-image-segmentation/train')
    mask_path = os.path.join(masks_base_dir, mask_relative_path)

    if os.path.exists(image_path) and os.path.exists(mask_path):
        display_image_with_mask(image_path, mask_path)

In [18]:
# Define the normalize_image function
def normalize_image(image):
    # Normalize the image to the range 0-255
    normalized_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized_image = clahe.apply(normalized_image)
    return equalized_image

Feature Extraction

In [19]:
def extract_features(image):
    rows, cols = image.shape
    # Compute gradients
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = cv2.magnitude(grad_x, grad_y)
    angle = cv2.phase(grad_x, grad_y, angleInDegrees=True)
    
    # Add texture features using Gabor filters
    gabor_kernels = [cv2.getGaborKernel((21, 21), 5, theta, 10, 0.5, 0, ktype=cv2.CV_32F) for theta in np.arange(0, np.pi, np.pi / 8)]
    gabor_features = [cv2.filter2D(image, cv2.CV_8UC3, kernel).flatten() for kernel in gabor_kernels]
    
    # Flatten the gradients and combine with Gabor features
    features = np.array([[r, c, image[r, c], magnitude[r, c], angle[r, c]] + [gabor[r * cols + c] for gabor in gabor_features] for r in range(rows) for c in range(cols)])
    return features
    
# Function to extract labels from masks
def extract_labels(mask):
    large_bowel_mask = mask[:,:,0]
    small_bowel_mask = mask[:,:,1]
    stomach_mask = mask[:,:,2]
    
    labels = np.zeros(large_bowel_mask.size, dtype=int)
    labels[large_bowel_mask.flatten() > 0] = 1
    labels[small_bowel_mask.flatten() > 0] = 2
    labels[stomach_mask.flatten() > 0] = 3
    
    return labels
    
# Collect features and labels from a subset of images and masks
features_list = []
labels_list = []
subset_size = 50  # choosing smaller subset for training

print("Extracting features and labels from images and masks...")
sampled_df = df.sample(subset_size, random_state=30)
for index, row in df.sample(subset_size).iterrows():
    image_relative_path = os.path.relpath(row['image_path'], '/kaggle/input/uw-madison-gi-tract-image-segmentation/train')
    image_path = os.path.join(images_base_dir, image_relative_path)

    mask_relative_path = os.path.relpath(row['mask_path'], '/kaggle/input/uwmgi-mask-dataset/png/uw-madison-gi-tract-image-segmentation/train')
    mask_path = os.path.join(masks_base_dir, mask_relative_path)

    if os.path.exists(image_path) and os.path.exists(mask_path):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path)
        
        if mask is None:
            continue

        image = normalize_image(image)

        # Split the mask into individual channels and check if all three masks contain non-zero values
        large_bowel_mask = mask[:,:,0]
        small_bowel_mask = mask[:,:,1]
        stomach_mask = mask[:,:,2]

        if not (np.any(large_bowel_mask) and np.any(small_bowel_mask) and np.any(stomach_mask)):
            continue
        
        features = extract_features(image)
        labels = extract_labels(mask)
        
        features_list.append(features)
        labels_list.append(labels)

# Combine all features and labels
features = np.vstack(features_list)
labels = np.hstack(labels_list)

print(f"Total features: {features.shape}")
print(f"Total labels: {labels.shape}")

Extracting features and labels from images and masks...
Total features: (182356, 13)
Total labels: (182356,)


Data Splitting

In [20]:
print("Data splitting...")
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=30)
print("Data split done")

Data splitting...
Data split done


Grid Search for the best K value using cross-validation

In [22]:
print("Finding best K...")
param_grid = {'n_neighbors': list(range(1, 10))}  
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=2, scoring='accuracy')  # 5-fold cross-validation
grid_search.fit(features_train, labels_train)

# Get the best K value
best_k = grid_search.best_params_['n_neighbors']
best_score = grid_search.best_score_

print(f"Best K value: {best_k}, with Cross-Validation Score: {best_score}")

Finding best K...
Best K value: 4, with Cross-Validation Score: 0.9715664500562091


Training

In [ ]:
print("Training K-Nearest Neighbors classifier...")
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(features_train, labels_train)
print("Model training completed")

In [ ]:
print("Saving images to folder...")
output_folder = '/Users/elizabethnemeti/Desktop/knn_results'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
def predict_and_display(image_path, count, display=False):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # normalize
    image = normalize_image(image)

    # feature extraction
    features = extract_features(image)
    
    #get label predicitions
    print(f"Predicting segmentation for {image_path}...")
    predicted_labels = knn.predict(features)
    
    # match labels to correct dimension
    segmented_image = predicted_labels.reshape(image.shape)
    
    # add color mask for viz 
    color_mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    color_mask[segmented_image == 1] = [0, 255, 0]  # Green for large bowel
    color_mask[segmented_image == 2] = [0, 0, 255]  # Red for small bowel
    color_mask[segmented_image == 3] = [255, 0, 0]  # Blue for stomach
    
    # overlay mask on image here
    overlay = cv2.addWeighted(cv2.merge([image, image, image]), 0.7, color_mask, 0.3, 0)

    output_file_path = os.path.join(output_folder, f'segmentation_{count}.png')
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title('Segmented Image')
    plt.imshow(overlay)
    plt.axis('off')

    # adding a legend
    legend_elements = [
        Patch(facecolor='red', edgecolor='r', label='Stomach'),
        Patch(facecolor='green', edgecolor='g', label='Large Bowel'),
        Patch(facecolor='blue', edgecolor='b', label='Small Bowel')
    ]
    plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.2, 1))
    
    plt.savefig(output_file_path, format='png')

    if display:
        plt.show()
    else:
        plt.close()

    # Clear variables to free memory
    del image, features, predicted_labels, segmented_image, color_mask, overlay
    import gc
    gc.collect()

Adds 200 images to folder

In [ ]:
count = 0
visualization_limit = 200

print("Predicting and displaying segmentation for new images...")
for index, row in df.iterrows():
    if count >= visualization_limit:
        break

    image_relative_path = os.path.relpath(row['image_path'], '/kaggle/input/uw-madison-gi-tract-image-segmentation/train')
    image_path = os.path.join(images_base_dir, image_relative_path)

    if os.path.exists(image_path):
        predict_and_display(image_path, count, display=(count == 0))
        count += 1

print("Segmentation process completed")